In [ ]:
#%pip install yfinance
#pip install sdmx1



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 3.4 MB/s eta 0:00:01
     ----------------- ---------------------- 1.3/3.0 MB 3.5 MB/s eta 0:00:01
     ------------------------------- -------- 2.4/3.0 MB 3.9 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 4.1 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pypro

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [123]:
#$env:PYTHONPATH="src"      # PowerShell
import sys
import os

project_root = os.path.abspath("../src")
sys.path.append(project_root)

%load_ext autoreload
%autoreload
from curves.yield_curve import YieldCurve
from products.coupon_bond import CouponBond
from products.future import EquityFuture
from products.swap import InterestRateSwap
from products.option import EuropeanCall
from models.black_scholes import BlackScholesModel
from models.calibration import ImpliedVolatilitySolver
from data_loader.yield_curve_loader import YieldCurveLoader
from data_loader.equity_loader import EquityLoader



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# les produits

In [3]:
curve = YieldCurve(
    maturities=[1, 2, 3, 5],
    zero_rates=[0.02, 0.025, 0.03, 0.035]
)

print(curve.zero_rate(4))   # interpolation entre 3y et 5y
print(curve.discount_factor(4))


0.0325
0.8780954309205613


In [4]:
bond = CouponBond(nominal=100, coupon_rate=0.05, maturity=5, payment_frequency=1)

print("Bond price =", bond.price(curve))

Bond price = 106.76026075076305


In [5]:
fut = EquityFuture(
    spot=4000,       # S&P 500 spot
    rate=0.03,       # 3%
    dividend_yield=0.015,  # 1.5%
    maturity=0.5     # 6 months
)

print("Future price =", fut.price())


Future price = 4030.1127817781353


In [6]:
swap = InterestRateSwap(
    notional=1_000_000,
    fixed_rate=0.03,
    payment_times=[1, 2, 3]
)

print("Fixed Leg PV =", swap.pv_fixed_leg(curve))
print("Floating Leg PV =", swap.pv_floating_leg(curve))
print("Swap Value =", swap.price(curve))
print("Swap rate =", swap.swap_rate(curve))

Fixed Leg PV = 85360.77849236093
Floating Leg PV = 86068.81472877182
Swap Value = 708.0362364108878
Swap rate = 0.02328967326889742


In [7]:
call = EuropeanCall(strike=100, maturity=1.0)
model = BlackScholesModel(spot=105, rate=0.03)

vol = 0.2

print("Call price =", model.call_price(call, vol))
print("Delta =", model.call_delta(call, vol))
print("Gamma =", model.gamma(call, vol))
print("Vega =", model.vega(call, vol))


Call price = 12.638755916496294
Delta = 0.6893295440636719
Gamma = 0.01681549194942603
Vega = 37.078159748484396


In [8]:
# Market price of the call
market_price = 12.50

# Solver
solver = ImpliedVolatilitySolver(model)

iv = solver.implied_vol_call(call, market_price)

print("Implied volatility =", iv)

Implied volatility = 0.19625254622411945


# data

In [ ]:
YC = YieldCurveLoader()
df = YC.get_rate(key='SR_20Y')
#YC.reload_yield_curve()

In [79]:
EQ = EquityLoader()
data = EQ.find('cac 40')
data.head()

,Symbole,Nom,Dernier cours,Secteur/Catégorie,Type,Change
0,^FCHI,CAC 40,"8 046,59",--,Indices,PAR
1,GC40.DE,AIS-AMUNDI CAC 40 ESG N,"141,82",--,Trackers,GER
2,CAC6L.PA,CAC 40 X6 Leverage NR,"2 937,65",--,Indices,PAR
3,LVC.PA,Amundi CAC 40 Daily (2x) Levera,"42,39",--,Trackers,PAR
4,CA10S.PA,CAC 40 X10 Short GR,"3 068,87",--,Indices,PAR


In [124]:
EQ = EquityLoader()
print(f' VIX : {EQ.get_VIX()}')
print(f' VCAC : {EQ.get_VCAC()}')

e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:80: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, end=end, period=period)
[*********************100%***********************]  1 of 1 completed
e:\Cours3A\Modèles de pricing des derivatives\asset-pricing\src\data_loader\equity_loader.py:96: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(data.iloc[-1]['Close']) / 100


 VIX : 0.18559999465942384
 VCAC : 1.0000000000000003e-05


In [125]:
EQ.get_implied_volatility('^FCHI')

np.float64(1.0000000000000003e-05)